# Calculating E(B-V) from NaID lines 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
path = os.getcwd()+'/'
naid = pd.read_csv(path+'spec/NaID.csv')
naid

,phase,line,pew,pewerr
0,12.3,D2,0.192083,0.064301
1,12.3,D1,0.079923,0.068859
2,28.4,D2,0.144041,0.028625
3,28.4,D1,0.066316,0.039559


In [3]:
def Poznanski2012(D1,D1err,D2,D2err):
    if D1 != 0:
        # using equation 8 from Poznanski 2012 for E(B-V)
        # log10(E(B-V)) = 2.47*EW(D1) - 1.76 +/- 0.17
        a = 2.47
        c = 1.75
        cerr = 0.17
        err = np.sqrt(D1err**2+cerr**2)

        ebv = 10**((a*(D1))-c)
        ebverr = 2.303*ebv*err
        print('D1 only:')
        print(round(ebv,3),'+/-',round(ebverr,3))
        
    if D2 != 0:
        # using equation 6 from Poznanski 2012 for E(B-V)
        # log10(E(B-V)) = 2.16*EW(D2) - 1.19 +/- 0.15
        a = 2.16
        c = 1.19
        cerr = 0.15
        err = np.sqrt(D2err**2+cerr**2)

        ebv = 10**((a*(D2))-c)
        ebverr = 2.303*ebv*err
        print('D2 only:')
        print(round(ebv,3),'+/-',round(ebverr,3))
    
    if D1 != 0 and D2 != 0:
        # using equation 9 from Poznanski 2012 for E(B-V)
        # log10(E(B-V)) = 1.17*EW(D1+D2) - 1.85 +/- 0.08
        a = 1.17
        c = 1.85
        cerr = 0.08
        err = np.sqrt(D1err**2+D2err**2+cerr**2)

        ebv = 10**((a*(D1+D2))-c)
        ebverr = 2.303*ebv*err
        print('D1+D2:')
        print(round(ebv,3),'+/-',round(ebverr,3))
        return [ebv,ebverr]
    
# error propagation for the different phases
def propagate(ebv,err):
    def f(f0,mag):
        return f0*10**(-mag/2.5)
    def m(flux,f0):
        return -2.5*np.log10(flux/f0)
    def df(flux,dm):
        return flux*dm*(-np.log(10)/2.5)
    def quad(errs):
        return np.sqrt(sum([x**2 for x in errs]))

    # convert to flux and then propogate the errors because mag is a log scale and this is easier
    f0 = 10**-9 # vega
    flux = [f(f0,m) for m in ebv]
    dflux = [df(flux[i],err[i]) for i in range(len(err))]
    avgf = sum(flux)/len(ebv)
    properr = quad(dflux)
    propdm = abs(-2.5/np.log(10)*(properr/avgf))
    avgebv = sum([m(x,f0) for x in flux])/len(flux)

    print(flux,avgf,dflux,properr,propdm,avgebv)
    
    return [avgebv,propdm]
        
ebv, err = [], []
for phase in set(naid.phase.values):
    print('spec phase:',phase)
    loopdf = naid[(naid.phase == phase)]
    D1 = loopdf[(loopdf.line == 'D1')]['pew'].values[0]
    D1err = loopdf[(loopdf.line == 'D1')]['pewerr'].values[0]
    D2 = loopdf[(loopdf.line == 'D2')]['pew'].values[0]
    D2err = loopdf[(loopdf.line == 'D2')]['pewerr'].values[0]
    lebv, lerr = Poznanski2012(D1,D1err,D2,D2err)
    ebv.append(lebv)
    err.append(lerr)
    print('---------------')
    
ebvFinal, errFinal = propagate(ebv,err)
print('---------------')
print(round(ebvFinal,3),round(errFinal,3))

spec phase: 12.3
D1 only:
0.028 +/- 0.012
D2 only:
0.168 +/- 0.063
D1+D2:
0.029 +/- 0.008
---------------
spec phase: 28.4
D1 only:
0.026 +/- 0.01
D2 only:
0.132 +/- 0.046
D1+D2:
0.025 +/- 0.005
---------------
[9.73290997996135e-10, 9.773314175903982e-10] 9.753112077932666e-10 [-7.500106617067366e-12, -4.837089706783884e-12] 8.924630866251685e-12 0.009935079970541334 0.02714429181934043
---------------
0.027 0.01


In [4]:
# total E(B-V)
ebvMW, ebverrMW = 0.0192, 0.0005
ebv = [ebvFinal,ebvMW]
ebverr = [errFinal,ebverrMW]
print('ebv:',round(sum(ebv),3))
print('ebv error:',round(propagate(ebv,ebverr)[1],3))

ebv: 0.046
[9.753091155113522e-10, 9.824715882051594e-10] 9.788903518582557e-10 [-8.924611720728075e-12, -4.5244488665827697e-13] 8.936073004470149e-12 0.009911445108123697 0.023172145909670185
ebv error: 0.01
